In [0]:
import dataiku
import pandas as pd

# Connect to the DSS instance
client = dataiku.api_client()


In [0]:
# Prepare a list to store results
project_hive_counts = []

# Iterate over all projects
projects = client.list_project_keys()
for project_key in projects:
    project = client.get_project(project_key)
    
    # Get project metadata to extract owner information
    project_metadata = project.get_metadata()
    
    # Get the owner - this will check the owner field in metadata
    owner = project_metadata.get('owner', '')
    
    # Get the owner's email - you might need to fetch from users API if available
    if owner:
        try:
            # Try to get user details from the client
            user_details = client.get_user(owner)
            owner_email = user_details.get('email', '')
        except:
            # If we can't get user details, use the username as fallback
            owner_email = f"{owner}@example.com"  # Fallback pattern, replace with actual domain
    else:
        owner_email = ""
    
    # Get the list of recipes in the project
    recipes = project.list_recipes()
    
    # Count the number of Hive recipes
    hive_count = sum(1 for recipe in recipes if recipe['type'] == 'Hive')

    # Append the result to the list
    project_hive_counts.append({
        'PROJECT_KEY': project_key, 
        'NUMBER_OF_HIVE_RECIPES_USED_IN_THIS_PROJECT': hive_count,
        'PROJECT_OWNER_EMAIL': owner_email
    })

# Create a Pandas DataFrame with the results
df = pd.DataFrame(project_hive_counts)

df.sort_values(by="hive_recipe_count", ascending=False)